In [1]:
import pandas as pd
my_books = pd.read_csv("liked_books.csv", index_col=0)

In [2]:
my_books

list = []
import csv
with open("liked_books.csv") as f:
    csv = csv.reader(f)
    for row in csv:
        if row[1] == "url":
            continue
        list.append({"user_id": -1, "book_id": row[1],"rating": 5})
list1 = pd.DataFrame.from_dict(list)
list1

,user_id,book_id,rating
0,-1,https://www.novelupdates.com/series/reverend-i...,5
1,-1,https://www.novelupdates.com/series/soul-of-ne...,5


In [3]:
 my_books["url"] = my_books["url"].astype(str)

In [4]:
book_set = set(my_books["url"])
book_set  

{'https://www.novelupdates.com/series/reverend-insanity/',
 'https://www.novelupdates.com/series/soul-of-negary/'}

In [5]:
def urlcorrector(string):
    if "#comments" in string:
        string = string.rsplit("/",1)[0]
        if "comment-page" in string:
            string = string.rsplit("/",1)[0]
        string = string + "/"
    return(string)


In [6]:
overlap_users = {}

import csv

with open("userinteractiondata.csv") as f:

    csv = csv.reader(f)
    
    for row in csv:
        url = urlcorrector(row[1])
        if url in book_set:
            if row[2] not in overlap_users:
                overlap_users[row[2]] = 1
            else:
                overlap_users[row[2]] = overlap_users[row[2]] + 1

len(overlap_users)

319

In [7]:
urlcorrector('https://www.novelupdates.com/series/martial-peak/#comments')

'https://www.novelupdates.com/series/martial-peak/'

In [8]:
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k]>my_books.shape[0]/15]  )

In [9]:
len(filtered_overlap_users)

319

In [10]:
import csv

interactions_list = []

with open("userinteractiondata.csv") as f:

    userinteraction = csv.reader(f)
     
    for row in userinteraction:
        if row[2] in filtered_overlap_users:
             interactions_list.append([row[2],row[1],row[4]])

In [11]:
interactions_list[0]

['https://www.novelupdates.com/user/96840/kkgoh/',
 'https://www.novelupdates.com/series/im-really-not-the-demon-gods-lackey/',
 '3']

In [12]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])
interactions = pd.concat([list1[["user_id", "book_id", "rating"]], interactions])
interactions

,user_id,book_id,rating
0,-1,https://www.novelupdates.com/series/reverend-i...,5
1,-1,https://www.novelupdates.com/series/soul-of-ne...,5
0,https://www.novelupdates.com/user/96840/kkgoh/,https://www.novelupdates.com/series/im-really-...,3
1,https://www.novelupdates.com/user/30989/Pixeld...,https://www.novelupdates.com/series/im-really-...,5
2,https://www.novelupdates.com/user/329670/Daois...,https://www.novelupdates.com/series/the-main-h...,5
...,...,...,...
7353,https://www.novelupdates.com/user/130085/Zeuso...,https://www.novelupdates.com/series/i-have-a-s...,0
7354,https://www.novelupdates.com/user/60924/SayMrrp/,https://www.novelupdates.com/series/my-hero/,5
7355,https://www.novelupdates.com/user/118695/Gener...,https://www.novelupdates.com/series/paladin-of...,5
7356,https://www.novelupdates.com/user/300368/Asphy...,https://www.novelupdates.com/series/paladin-of...,5


In [13]:
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])

In [14]:
interactions["user_id"].unique()

array(['-1', 'https://www.novelupdates.com/user/96840/kkgoh/',
       'https://www.novelupdates.com/user/30989/Pixeldrum/',
       'https://www.novelupdates.com/user/329670/Daoist Ghoul/',
       'https://www.novelupdates.com/user/324144/ISuckAtWriting/',
       'https://www.novelupdates.com/user/29732/AvERaGeNP/',
       'https://www.novelupdates.com/user/52256/Davr/',
       'https://www.novelupdates.com/user/95898/Queen of Sheba/',
       'https://www.novelupdates.com/user/28113/Demonic Reader lv 451F/',
       'https://www.novelupdates.com/user/322878/Alex Woods/',
       'https://www.novelupdates.com/user/4456/Beel/',
       'https://www.novelupdates.com/user/966/ninthlite/',
       'https://www.novelupdates.com/user/58244/Seregosa/',
       'https://www.novelupdates.com/user/14695/bushwhacker2k/',
       'https://www.novelupdates.com/user/76750/Gsichtselfmeter/',
       'https://www.novelupdates.com/user/46035/Mahiro9/',
       'https://www.novelupdates.com/user/13287/Aziru/',
  

In [15]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes

In [16]:
interactions.iloc[0]

user_id                                                      -1
book_id       https://www.novelupdates.com/series/reverend-i...
rating                                                        5
user_index                                                    0
Name: 0, dtype: object

In [17]:
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

In [18]:
from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"],interactions["book_index"])))

In [19]:
ratings_mat_coo.shape

(320, 3628)

In [20]:
ratings_mat = ratings_mat_coo.tocsr()

In [21]:
interactions[interactions["user_id"]=="-1"]
my_index = 0

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [23]:
similarity[0]

0.9999999999999998

In [24]:
import numpy as np

indices = np.argpartition(similarity, -20)[-20:]

In [25]:
indices

array([313, 171, 150, 287, 255, 270,  66,  48, 213, 108, 251, 156,  60,
        21, 264,  16, 146, 290, 318,   0], dtype=int64)

In [26]:
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

In [27]:
similar_users = similar_users[similar_users["user_id"]!="-1"]
similar_users

,user_id,book_id,rating,user_index,book_index
6,https://www.novelupdates.com/user/95898/Queen ...,https://www.novelupdates.com/series/world-of-c...,4,313,3541
10,https://www.novelupdates.com/user/4456/Beel/,https://www.novelupdates.com/series/peerless-m...,5,251,2032
113,https://www.novelupdates.com/user/154209/Hellu...,https://www.novelupdates.com/series/solo-level...,3,48,2471
134,https://www.novelupdates.com/user/95898/Queen ...,https://www.novelupdates.com/series/world-of-c...,4,313,3541
141,https://www.novelupdates.com/user/4456/Beel/,https://www.novelupdates.com/series/peerless-m...,5,251,2032
...,...,...,...,...,...
6933,https://www.novelupdates.com/user/34276/adsegm/,https://www.novelupdates.com/series/eight-deso...,2,213,633
7019,https://www.novelupdates.com/user/459951/Tasmi/,https://www.novelupdates.com/series/waiting-un...,5,255,3463
7143,https://www.novelupdates.com/user/95898/Queen ...,https://www.novelupdates.com/series/dan-martia...,3,313,468
7168,https://www.novelupdates.com/user/95898/Queen ...,https://www.novelupdates.com/series/the-ultima...,4,313,3201


In [28]:
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [29]:
book_recs

,count,mean
book_id,,
https://www.novelupdates.com/series/9-heavenly-thunder-manual/,2,1.0
https://www.novelupdates.com/series/a-demons-path/,1,1.0
https://www.novelupdates.com/series/a-neets-guide-to-the-parallel-world-healer-the-strongest-cheat/,2,1.0
https://www.novelupdates.com/series/a-will-eternal/comment-page-2/#comments,1,3.0
https://www.novelupdates.com/series/abyss-domination/comment-page-2/#comments,1,4.0
...,...,...
https://www.novelupdates.com/series/world-of-cultivation/,2,4.0
https://www.novelupdates.com/series/world-of-warcraft-foreign-realm-domination/,2,1.0
https://www.novelupdates.com/series/worlds-apocalypse-online/comment-page-3/#comments,1,4.0


In [30]:
books_titles = pd.read_json("books_titles.json")
books_titles["url"] = books_titles["url"].astype(str)
books_titles

,url,title,ratings,cover,reduced_titles
0,https://www.novelupdates.com/series/taming-the...,Taming The Villainesses,183,https://cdn.novelupdates.com/images/2022/06/Ta...,tamingthevillainesses
1,https://www.novelupdates.com/series/the-main-h...,The Main Heroines are Trying to Kill Me,428,https://cdn.novelupdates.com/images/2022/04/Th...,themainheroinesaretryingtokillme
2,https://www.novelupdates.com/series/the-regres...,The Regressed Demon Lord is Kind,529,https://cdn.novelupdates.com/images/2021/06/Th...,theregresseddemonlordiskind
3,https://www.novelupdates.com/series/kidnapped-...,Kidnapped Dragons,683,https://cdn.novelupdates.com/images/2020/10/Ki...,kidnappeddragons
4,https://www.novelupdates.com/series/im-really-...,I’m Really Not The Demon God’s Lackey,509,https://cdn.novelupdates.com/images/2021/06/Im...,imreallynotthedemongodslackey
...,...,...,...,...,...
11318,https://www.novelupdates.com/series/little-ger...,"Little Ger, Slow Life in Another World",69,https://cdn.novelupdates.com/images/2021/10/Li...,littlegerslowlifeinanotherworld
11319,https://www.novelupdates.com/series/sword-and-...,Sword and Love,24,https://cdn.novelupdates.com/images/2018/09/Sw...,swordandlove
11320,https://www.novelupdates.com/series/misunderst...,Misunderstood Lousy Cop: Getting Promoted for ...,17,https://www.novelupdates.com/img/noimagefound.jpg,misunderstoodlousycopgettingpromotedforrunning...
11321,https://www.novelupdates.com/series/the-queens...,The Queen’s Husband,37,https://cdn.novelupdates.com/images/2019/03/tq...,thequeenshusband


In [31]:
books_recs = book_recs.merge(books_titles, how="inner",left_on ="book_id",right_on ="url")

In [32]:
books_recs

,count,mean,url,title,ratings,cover,reduced_titles
0,2,1.0,https://www.novelupdates.com/series/9-heavenly...,9 Heavenly Thunder Manual,338,https://cdn.novelupdates.com/images/2015/07/9m...,9heavenlythundermanual
1,1,1.0,https://www.novelupdates.com/series/a-demons-p...,A Demon’s Path,73,https://cdn.novelupdates.com/images/2017/11/11...,ademonspath
2,2,1.0,https://www.novelupdates.com/series/a-neets-gu...,"A NEET’s Guide to the Parallel World: Healer, ...",173,https://cdn.novelupdates.com/images/2019/11/bq...,aneetsguidetotheparallelworldhealerthestronges...
3,2,1.0,https://www.novelupdates.com/series/ankoku-kis...,Ankoku Kishi Monogatari ~Yuusha wo Taosu Tamen...,436,https://cdn.novelupdates.com/images/2020/05/an...,ankokukishimonogatariyuushawotaosutamenimaouni...
4,1,1.0,https://www.novelupdates.com/series/archfiend/,Archfiend,130,https://cdn.novelupdates.com/images/2017/11/st...,archfiend
...,...,...,...,...,...,...,...
68,1,3.0,https://www.novelupdates.com/series/when-god-m...,When God Made Me,141,https://cdn.novelupdates.com/images/2017/01/Is...,whengodmademe
69,2,5.0,https://www.novelupdates.com/series/wicked-sol...,Wicked Soldier King,251,https://cdn.novelupdates.com/images/2015/07/wi...,wickedsoldierking
70,2,1.0,https://www.novelupdates.com/series/world-of-w...,World of Warcraft: Foreign Realm Domination,423,https://cdn.novelupdates.com/images/2017/07/wo...,worldofwarcraftforeignrealmdomination
71,2,5.0,https://www.novelupdates.com/series/you-cannot...,You Cannot Afford To Offend My Woman,270,https://cdn.novelupdates.com/images/2019/03/yc...,youcannotaffordtooffendmywoman


In [33]:
books_recs["ratings"] = books_recs["ratings"].astype(int)
books_recs["adjusted_count"] = books_recs["count"]**(books_recs["count"] / books_recs["ratings"])

In [34]:
books_recs["score"] = books_recs["mean"]*books_recs["adjusted_count"]
books_recs = books_recs[~books_recs["url"].isin(my_books["url"])]

In [35]:
my_books["reduced_titles"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

In [36]:
my_books["reduced_titles"] = my_books["reduced_titles"].str.replace("\s+"," ", regex=True) 

In [37]:
books_recs = books_recs[~books_recs["reduced_titles"].isin(my_books["reduced_titles"])]

In [38]:
books_recs = books_recs[books_recs["count"]>1]
books_recs = books_recs[books_recs["mean"]>1]
top_recs = books_recs.sort_values("score", ascending=False)

In [39]:
top_recs

,count,mean,url,title,ratings,cover,reduced_titles,adjusted_count,score
40,2,5.0,https://www.novelupdates.com/series/power-and-...,Power and Wealth,103,https://cdn.novelupdates.com/images/2019/03/co...,powerandwealth,1.013550,5.067751
34,2,5.0,https://www.novelupdates.com/series/my-senior-...,My Senior Brother is Too Steady,174,https://cdn.novelupdates.com/images/2020/08/sb...,myseniorbrotheristoosteady,1.007999,5.039995
52,2,5.0,https://www.novelupdates.com/series/tales-of-t...,Tales Of The World Devouring Serpent,189,https://cdn.novelupdates.com/images/2020/06/wo...,talesoftheworlddevouringserpent,1.007362,5.036809
50,2,5.0,https://www.novelupdates.com/series/stealing-t...,Stealing The Heavens,190,https://cdn.novelupdates.com/images/2017/09/St...,stealingtheheavens,1.007323,5.036615
11,2,5.0,https://www.novelupdates.com/series/death-sutra/,Death Sutra,214,https://cdn.novelupdates.com/images/2017/01/t7...,deathsutra,1.006499,5.032495
27,2,5.0,https://www.novelupdates.com/series/immortal/,Immortal,248,https://cdn.novelupdates.com/images/2018/04/im...,immortal,1.005606,5.028028
69,2,5.0,https://www.novelupdates.com/series/wicked-sol...,Wicked Soldier King,251,https://cdn.novelupdates.com/images/2015/07/wi...,wickedsoldierking,1.005538,5.027692
16,2,5.0,https://www.novelupdates.com/series/everlastin...,Everlasting Immortal Firmament,255,https://cdn.novelupdates.com/images/2016/05/Ev...,everlastingimmortalfirmament,1.005451,5.027256
71,2,5.0,https://www.novelupdates.com/series/you-cannot...,You Cannot Afford To Offend My Woman,270,https://cdn.novelupdates.com/images/2019/03/yc...,youcannotaffordtooffendmywoman,1.005148,5.025738
64,2,5.0,https://www.novelupdates.com/series/the-strong...,The Strongest Gene,312,https://cdn.novelupdates.com/images/2017/05/42...,thestrongestgene,1.004453,5.022266


In [40]:
def make_clickable(val):
    return '<a target="_blank" href="{}">NovelUpdates</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover': show_image})

,count,mean,url,title,ratings,cover,reduced_titles,adjusted_count,score
40,2,5.000000,NovelUpdates,Power and Wealth,103,,powerandwealth,1.013550,5.067751
34,2,5.000000,NovelUpdates,My Senior Brother is Too Steady,174,,myseniorbrotheristoosteady,1.007999,5.039995
52,2,5.000000,NovelUpdates,Tales Of The World Devouring Serpent,189,,talesoftheworlddevouringserpent,1.007362,5.036809
50,2,5.000000,NovelUpdates,Stealing The Heavens,190,,stealingtheheavens,1.007323,5.036615
11,2,5.000000,NovelUpdates,Death Sutra,214,,deathsutra,1.006499,5.032495
27,2,5.000000,NovelUpdates,Immortal,248,,immortal,1.005606,5.028028
69,2,5.000000,NovelUpdates,Wicked Soldier King,251,,wickedsoldierking,1.005538,5.027692
16,2,5.000000,NovelUpdates,Everlasting Immortal Firmament,255,,everlastingimmortalfirmament,1.005451,5.027256
71,2,5.000000,NovelUpdates,You Cannot Afford To Offend My Woman,270,,youcannotaffordtooffendmywoman,1.005148,5.025738
64,2,5.000000,NovelUpdates,The Strongest Gene,312,,thestrongestgene,1.004453,5.022266
